In [2]:
import cv2
import os
import numpy as np

from PIL import Image
from glob import glob

In [3]:
img_dir = "/ontap/jsh/volume/datasets/imagenet-sample"
black_list = []
resolutions = {
    "sd": (854, 480),
    "hd": (1280, 720),
    "fhd": (1920, 1080),
    "qhd": (2560, 1440),
    "uhd": (3840, 2160),
}

for res in resolutions.keys():
    h = 0
    w = 0
    img_cnt = 0
    npy_dir = f"/ontap/jsh/volume/datasets/imagenet-sample-numpy-{res}"

    for tv in ["train", "val"]:
        labels = []
        os.makedirs(f"{npy_dir}/{tv}", exist_ok=True)

        for idx, img_path in enumerate(glob(f"{img_dir}/{tv}/**/*.JPEG")):
            if img_path in black_list:
                continue

            # Read image
            img = cv2.imread(img_path)

            # Upsacle image
            upscaled_img = cv2.resize(
                img,
                dsize=(resolutions[res][0], resolutions[res][1]),
                interpolation=cv2.INTER_LINEAR,
            )

            # Image to array
            img_array = np.asarray(upscaled_img)
            # img_array = np.moveaxis(img_array, -1, 0)  # fn.crop_mirror_normalize 인자로 output_layout="CHW"를 주면 됨
            # 흑백 이미지 or 가로 또는 세로가 224 미만인 이미지 스킵
            if len(img_array.shape) == 2 or img_array.shape[0] < 224 or img_array.shape[1] < 224:
                black_list.append(img_path)
                continue
            file = img_path.split("/")[-1]
            file = file.replace("JPEG", "image.npy")
            np.save(f"{npy_dir}/{tv}/{file}", img_array)

            # Label
            label = img_path.split("/")[-2]
            if not label in labels:
                labels.append(label)
            file = file.replace("image.npy", "label.npy")
            np.save(f"{npy_dir}/{tv}/{file}", np.array([len(labels)]))

            # For calculate resolution
            h += img_array.shape[0]
            w += img_array.shape[1]
            img_cnt += 1

    print(f"{res}: {int(w/img_cnt)} x {int(h/img_cnt)}")

sd: 854 x 480
hd: 1280 x 720
fhd: 1920 x 1080
qhd: 2560 x 1440
uhd: 3840 x 2160


In [1]:
import time
from nvidia.dali import pipeline_def, fn

In [2]:
@pipeline_def(batch_size=1, num_threads=3, device_id=0)
def create_pipe(device, tv):
    images = fn.readers.numpy(
        device=device,
        file_root=f"/gpfs/jsh/volume/datasets/imagenet-sample-numpy/{tv}",
        file_filter="*.image.npy",
        name="ImageReader",
    )
    labels = fn.readers.numpy(
        device=device,
        file_root=f"/gpfs/jsh/volume/datasets/imagenet-sample-numpy/{tv}",
        file_filter="*.label.npy",
        name="LabelReader",
    )
    return images, labels


def run_pipe(device, tv, idx):
    pipe = create_pipe(device, tv)
    pipe.build()
    data_size = list(pipe.epoch_size().values())[0]
    start_time = time.time()
    for _ in range(data_size):
        pipe_out = pipe.run()
        # if device =="cpu":
        #     img = pipe_out[0].as_array()
        #     label = pipe_out[1].as_array()
        # else:
        #     img = pipe_out[0].as_cpu().as_array()
        #     label = pipe_out[1].as_cpu().as_array()
    print(f"elapsed time [{device}-{tv}-{idx+1}]: {time.time() - start_time}")


for idx in range(1):
    device = "gpu"
    tv = "train"
    run_pipe(device, tv, idx)
    tv = "val"
    run_pipe(device, tv, idx)

    device = "cpu"
    tv = "train"
    run_pipe(device, tv, idx)
    tv = "val"
    run_pipe(device, tv, idx)

elapsed time [gpu-train-1]: 10.803786516189575
elapsed time [gpu-val-1]: 0.39374303817749023
elapsed time [cpu-train-1]: 9.387659311294556
elapsed time [cpu-val-1]: 0.37669920921325684


In [3]:
@pipeline_def(batch_size=1, num_threads=3, device_id=0)
def create_pipe(device, tv):
    images = fn.readers.numpy(
        device=device,
        file_root=f"/ontap/jsh/volume/datasets/imagenet-sample-numpy/{tv}",
        file_filter="*.image.npy",
        name="ImageReader",
    )
    labels = fn.readers.numpy(
        device=device,
        file_root=f"/ontap/jsh/volume/datasets/imagenet-sample-numpy/{tv}",
        file_filter="*.label.npy",
        name="LabelReader",
    )
    return images, labels


def run_pipe(device, tv, idx):
    pipe = create_pipe(device, tv)
    pipe.build()
    data_size = list(pipe.epoch_size().values())[0]
    start_time = time.time()
    for _ in range(data_size):
        pipe_out = pipe.run()
        # if device =="cpu":
        #     img = pipe_out[0].as_array()
        #     label = pipe_out[1].as_array()
        # else:
        #     img = pipe_out[0].as_cpu().as_array()
        #     label = pipe_out[1].as_cpu().as_array()
    print(f"elapsed time [{device}-{tv}-{idx+1}]: {time.time() - start_time}")


for idx in range(1):
    device = "gpu"
    tv = "train"
    run_pipe(device, tv, idx)
    tv = "val"
    run_pipe(device, tv, idx)

    device = "cpu"
    tv = "train"
    run_pipe(device, tv, idx)
    tv = "val"
    run_pipe(device, tv, idx)

elapsed time [gpu-train-1]: 10.365849256515503
elapsed time [gpu-val-1]: 0.7734766006469727
elapsed time [cpu-train-1]: 3.832213878631592
elapsed time [cpu-val-1]: 0.15192127227783203
